## Homework 3
### BIOSTAT 257
#### Joanna Boland
#### May 15, 2020

## Question 1: Problem Structure 

Let $\mathbf{A} \in \{0,1\}^{n \times n}$ be the connectivity matrix of $n$ web pages with entries,
\begin{eqnarray*}
    a_{ij}= \begin{cases},
    1 & \text{if page $i$ links to page $j$} \\,
    0 & \text{otherwise}
    \end{cases}.
\end{eqnarray*}
$r_i = \sum_j a_{ij}$ is the out-degree of page $i$. That is $r_i$ is the number of links on page $i$. Imagine a random surfer exploring the space of pages according to the following rules, 
* From a page $i$ with $r_i>0$, with probability $p$, (s)he randomly chooses a link on page $i$ (uniformly) and follows that link to the next page 
* with probability $1-p$, (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page, 
* From a page $i$ with $r_i=0$ (a dangling page), (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page, 

The process defines a Markov chain on the space of $n$ pages. Write the transition matrix $\mathbf{P}$ of the Markov chain as a sparse matrix plus rank 1 matrix.

We can write this in mathematical form as

$$
\begin{eqnarray*}
    p_{ij} = \begin{cases}
    \frac{1}{r_i}p + \frac{1}{n}(1 - p) & r_i > 0 \\
    \frac{1}{n} & r_i = 0
    \end{cases}
\end{eqnarray*}
$$

If we let $1$ be a vector of ones and $r$ is the vector containing all the $r_i$ values, then we can write $P$ as a matrix that is a sparse matrix plus rank 1 matrix as

$$P = \text{diag}\Big(\frac{p}{r}\Big) + \frac{1-p}{n}11^T$$

where $\frac{p}{r} = 0$ when $r_i = 0$. 

## Question 2: Relate to Numerical Linear Algebra

According to standard Markov chain theory, the (random) position of the surfer converges to the stationary distribution $\mathbf{x} = (x_1,\ldots,x_n)^T$ of the Markov chain. $x_i$ has the natural interpretation of the proportion of times the surfer visits page $i$ in the long run. Therefore $\mathbf{x}$ serves as page ranks: a higher $x_i$ means page $i$ is more visited. It is well-known that $\mathbf{x}$ is the left eigenvector corresponding to the top eigenvalue 1 of the transition matrix $\mathbf{P}$. That is $\mathbf{P}^T \mathbf{x} = \mathbf{x}$. Therefore $\mathbf{x}$ can be solved as an **eigen-problem**. It can also be cast as **solving a linear system**. Since the row sums of $\mathbf{P}$ are 1, $\mathbf{P}$ is rank deficient. We can replace the first equation by the $\sum_{i=1}^n x_i = 1$.

Hint: For iterative solvers, we don't need to replace the 1st equation. We can use the matrix $\mathbf{I} - \mathbf{P}^T$ directly if we start with a vector with all positive entries.


## Question 3: Explore the Data

In [1]:
using MatrixDepot, SparseArrays, LinearAlgebra, UnicodePlots

md = mdopen("SNAP/web-Google")
# display documentation for the SNAP/web-Google data

# connectivity matrix
A = md.A

include group.jl for user defined matrix generators
verify download of index files...
used remote site is https://sparse.tamu.edu/?per_page=All


916428×916428 SparseMatrixCSC{Bool,Int64} with 5105039 stored entries:
  [11343 ,      1]  =  1
  [11928 ,      1]  =  1
  [15902 ,      1]  =  1
  [29547 ,      1]  =  1
  [30282 ,      1]  =  1
  [31301 ,      1]  =  1
  [38717 ,      1]  =  1
  [43930 ,      1]  =  1
  [46275 ,      1]  =  1
  [48193 ,      1]  =  1
  [50823 ,      1]  =  1
  [56911 ,      1]  =  1
  ⋮
  [608625, 916427]  =  1
  [618730, 916427]  =  1
  [622998, 916427]  =  1
  [673046, 916427]  =  1
  [716616, 916427]  =  1
  [720325, 916427]  =  1
  [772226, 916427]  =  1
  [785097, 916427]  =  1
  [788476, 916427]  =  1
  [822938, 916427]  =  1
  [833616, 916427]  =  1
  [417498, 916428]  =  1
  [843845, 916428]  =  1

* There are 3 bits in a single digit, so if we had a matrix of 916428 x 916428 entries of zeros or 1, then the number of gigabytes of that matrix would be:

In [2]:
GB = ((916428)^2 * 3)/(8000000000)

314.940104694

* The number of web pages is $916,428$ as the A matrix is and $n \times n$ matrix where $n = $ total number of web page entries
* Since the only stored entries are 1's and the rest are assumed to be 0's, then there are $5,105,039$ web links.

In [3]:
out = vec(sum(A, dims = 1)) # out-degrees
x = vec(sum(A, dims = 2)) # in-degrees
count(iszero, out)

201883

* The number of dangling nodes is $201,883$.

In [4]:
histogram(x, nbins = 25, closed = :left, title = "Histogram of In-Degrees")

                           Histogram of In-Degrees
                  ┌                                        ┐ 
   [  0.0,  20.0) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 891798   
   [ 20.0,  40.0) ┤▇ 22628                                   
   [ 40.0,  60.0) ┤ 1329                                     
   [ 60.0,  80.0) ┤ 371                                      
   [ 80.0, 100.0) ┤ 124                                      
   [100.0, 120.0) ┤ 86                                       
   [120.0, 140.0) ┤ 29                                       
   [140.0, 160.0) ┤ 15                                       
   [160.0, 180.0) ┤ 15                                       
   [180.0, 200.0) ┤ 8                                        
   [200.0, 220.0) ┤ 7                                        
   [220.0, 240.0) ┤ 3                                        
   [240.0, 260.0) ┤ 8                                        
   [260.0, 280.0) ┤ 3                                        
   [280.0, 300.0) ┤

* The top 20 pages with the largest in-degrees are:

In [19]:
sortperm(vec(x), rev = true)[1:20]

20-element Array{Int64,1}:
 506743
 203749
 305230
 768092
 808644
 412411
 600480
 376429
 156951
 885729
 667585
 685696
 282141
 598189
 579315
 411594
 321092
 838279
 302734
 915274

In [5]:
histogram(out, nbins = 25, closed = :left, title = "Histogram of Out-Degrees")

                            Histogram of Out-Degrees
                    ┌                                        ┐ 
   [   0.0,  500.0) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 916114   
   [ 500.0, 1000.0) ┤ 180                                      
   [1000.0, 1500.0) ┤ 32                                       
   [1500.0, 2000.0) ┤ 20                                       
   [2000.0, 2500.0) ┤ 16                                       
   [2500.0, 3000.0) ┤ 20                                       
   [3000.0, 3500.0) ┤ 18                                       
   [3500.0, 4000.0) ┤ 10                                       
   [4000.0, 4500.0) ┤ 9                                        
   [4500.0, 5000.0) ┤ 4                                        
   [5000.0, 5500.0) ┤ 4                                        
   [5500.0, 6000.0) ┤ 0                                        
   [6000.0, 6500.0) ┤ 1                                        
                    └                              

* The top 20 pages with the largest number of out-degrees are:


In [20]:
sortperm(vec(x), rev = true)[1:20]

20-element Array{Int64,1}:
 506743
 203749
 305230
 768092
 808644
 412411
 600480
 376429
 156951
 885729
 667585
 685696
 282141
 598189
 579315
 411594
 321092
 838279
 302734
 915274

In [6]:
A1 = A[1:10000, 1:10000]
spy(A1, title = "Sparsity of a Submatrix of A")

                Sparsity of a Submatrix of A
         ┌──────────────────────────────────────────┐    
       1 │⠀⠀⠀⠀⠀⠀⠀⠂⠀⠀⠀⠀⢀⢂⡂⠐⠀⠀⠀⠠⠀⠀⠀⠔⠄⠀⠈⢀⠀⠀⠉⠀⠀⠁⠀⠀⠐⠀⠄⠀⠐⠀│ > 0
         │⠀⠀⠀⠀⠀⠂⡂⠀⠄⠄⠈⠈⡔⠀⠀⠁⠂⠀⠀⠀⠀⠠⠁⠒⠁⠀⠄⠀⠐⠀⠈⠀⠀⡐⠀⠀⠠⠀⡀⠀⠉⠀│ < 0
         │⠀⠐⠠⠀⠀⠁⢐⠄⠀⠢⠠⠀⠀⡀⠈⠀⠋⠀⠀⠀⠀⠀⠂⡀⠀⠈⠄⠀⠄⠁⠈⠀⠠⠀⠰⠀⠀⠀⠀⢀⠆⠀│    
         │⠀⠈⠠⠂⠀⠈⠀⡠⠐⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⠈⠄⠁⢀⡀⠐⠄⠄⠀⠀⢀⠀⠈⠂⢠⠀⠢⠐⠀⠀⠀⡀│    
         │⠀⠀⠐⡁⠀⡀⠀⠀⠠⠀⠁⠀⠀⠀⡄⠀⠈⠁⠀⠀⠀⠀⠠⠂⠀⠀⠀⠀⠀⠂⠪⠀⠀⠂⠀⠀⠀⠄⢀⠀⠁⠀│    
         │⠂⢀⠀⠁⠀⠀⡀⡉⠀⠀⠠⠂⠐⢀⠄⠁⠐⠀⠈⠀⠀⠀⠀⠀⡀⠐⠁⡠⠀⠂⠚⠀⠁⠀⠐⢀⠆⠀⠈⠄⢀⠂│    
         │⠀⠀⠀⠁⠀⡄⠀⠀⠀⠀⠒⠀⢄⠄⢂⠀⠂⠀⠀⠄⠀⢀⠀⠣⠀⢀⠀⠀⠀⠀⠠⡀⠀⠀⢠⠄⠀⡀⠀⠂⠂⠀│    
         │⢀⠈⠀⠀⠂⠀⠀⠁⢠⠈⠄⠁⠀⠀⡈⠈⠠⠀⢀⡦⠀⠀⠅⠐⠀⠨⠁⠨⠀⠄⠠⢀⠀⠐⢐⠀⠀⠉⠀⠀⠠⠉│    
         │⠀⠀⠨⠀⠋⠀⠄⠀⠂⠀⠈⠀⠀⠀⠈⠀⠀⠀⠀⡑⠀⢀⠈⠈⠀⠀⠀⠈⠀⠂⠐⠼⠀⠠⠀⠀⡈⠁⠀⠀⠡⠄│    
         │⠠⡁⠀⠀⠀⠀⠀⠀⠀⠀⠂⢀⠀⠀⠀⡄⠀⠀⠀⠤⠀⠀⠁⠀⡂⠀⠀⠀⠄⠀⢈⠀⢔⡊⣨⠈⠂⠀⠄⢀⠀⠀│    
         │⠀⠂⠀⠈⠀⠀⠀⠀⠀⠀⠈⠀⡀⠀⠂⠠⠐⠀⠄⠀⠀⠀⠀⠀⠀⠤⠀⠀⠀⠠⠘⠐⠀⠄⢘⠀⠀⢐⠄⠀⠁⠀│    
         │⢀⠄⢠⠄⡊⠂⠢⢀⠀⠀⠀⠀⠄⡀⡑⠀⡐⠀⠁⡂⠀⠀⠀⠰⠂⢠⠄⠈⠀⠈⠐⠀⠀⠀⠉⠀⠀⡀⠀⠀⠃⠀│    
         │⠀⠀⠅⠀⡁⠀⠀⠀⠀⠀⢁⢀⠀⡀⡀⡐⠀⠀⠈⠈⠔⠄⠀⠀⢀⠀⠀⠀⠀⠀⠀⠀⠀⠄⢉⠠⠁⠀⠁⠀⠀⠀│    
         │⠊⢈⠀⠁⠀⠀⠀⠅⠀⠂⠀⡀⠀⠀⠀⠀⡀⠀⠀⠐⠄⠁⠠⠀⠐⠈⠀⠀⠀⡀⢠⠀⠀⠀⢰⠀⠀⠀⢀⠀⠠⠂│    
         │⠈⠀⠀⠈⠄⠀⠀⠄⠐⠀⠠⠁⠁⠀⡂⠀⠠⠄⠌⠅⣀⠀⠀⢀⠀⠠⠀⠠⠠⠀⠲⢂⠀⠰⡄⠀⢀⠬⠈⠀⠀⠀│    
         │⠈⠀⠀⠄⠀⠀⡀⠈⠀⠀⠙⠀⠀⠀⢀⢀⠂

## Question 4: Dense Linear Algebra

Consider the following methods to obtain the page ranks of the `SNAP/web-Google` data. 

1. A dense linear system solver such as LU decomposition.
2. A dense eigen-solver for asymmetric matrix.

For the LU approach, estimate (1) the memory usage and (2) how long it will take assuming that the LAPACK functions can achieve the theoretical throughput of your computer.

The GE/LU approach will take
$$2 \times (109)^3/3/10^{12}≈6.66 \text{ TB}$$

While the amount of time it would take is

$$6.66×10^{14} \text{seconds} ≈20  \text{ million years}$$

## Question 5: Iterative Solvers

Set the _teleportation_ parameter at $p = 0.85$. Consider the following methods for solving the PageRank problem. 

1. An iterative linear system solver such as GMRES.
2. An iterative eigen-solver such as Arnoldi method.

For iterative methods, we have many choices in Julia. See a list of existing Julia packages for linear solvers at this [page](https://jutho.github.io/KrylovKit.jl/stable/#Package-features-and-alternatives-1). The start-up code below uses the [KrylovKit.jl](https://github.com/Jutho/KrylovKit.jl) package. You can use other packages if you prefer. Make sure to utilize the special structure of $\mathbf{P}$ (sparse + rank 1) to speed up the matrix-vector multiplication.


### Step 1

Let's implement a type `PageRankImPt` that mimics the matrix $\mathbf{M} = \mathbf{I} - \mathbf{P}^T$. For iterative methods, all we need to provide is methods for evaluating $\mathbf{M} \mathbf{v}$ and $\mathbf{M}^T \mathbf{v}$ for arbitrary vector $\mathbf{v}$.

In [7]:
using BenchmarkTools, LinearAlgebra, SparseArrays, Revise

# a type for the matrix M = I - P^T in PageRank problem
struct PageRankImPt{TA <: Number, IA <: Integer, T <: AbstractFloat} <: AbstractMatrix{T}
    A         :: SparseMatrixCSC{TA, IA} # adjacency matrix
    telep     :: T
    r         :: Vector{T}
    d         :: Vector{T}
    z         :: Vector{T} 
    dtv       :: Vector{T}
    AtDtvb    :: Vector{T}
    o1dtv     :: Vector{T} 
    Av        :: Vector{T}
    DAv       :: Vector{T}
    ztonetv   :: Vector{T}
    one       :: Vector{T}
end

# constructor
function PageRankImPt(A::SparseMatrixCSC, telep::T) where T <: AbstractFloat
    n = size(A, 1)
    one = ones(n)
    r = A * one
    d = Vector{Float64}(undef, n)
    z = Vector{Float64}(undef, n)
    dtv = Vector{Float64}(undef, n)
    AtDtvb = Vector{Float64}(undef, n)
    o1dtv = Vector{Float64}(undef, n)
    Av = Vector{Float64}(undef, n)
    DAv = Vector{Float64}(undef, n)
    ztonetv = Vector{Float64}(undef, n)
    for i in 1:n
        if r[i] == 0
            d[i], z[i] = 0, 1 / n
        else 
            d[i], z[i] = telep / r[i], (1 - telep) / n
        end
    end
    PageRankImPt(A, telep, r, d, z, dtv, AtDtvb, o1dtv, Av, DAv, ztonetv, one)
end

LinearAlgebra.issymmetric(::PageRankImPt) = false
Base.size(M::PageRankImPt) = size(M.A)

function Base.getindex(M::PageRankImPt, i, j) 
    if i == j
        outsc = 1 - M.A[j, i] * M.d[i] - M.z[i]
    else 
        outsc = - M.A[j, i] * M.d[i] - M.z[j]
    end
    return outsc
end 

# overwrite `out` by `(I - Pt) * v`
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        M   :: PageRankImPt{<:Number, <:Integer, T}, 
        v   :: Vector{T}) where T <: AbstractFloat
    M.dtv .= M.d .* v
    ztv = dot(M.z, v)
    mul!(M.AtDtvb, transpose(M.A), M.dtv)
    M.o1dtv .= M.one .* ztv
    axpby!(-1, M.AtDtvb, -1, M.o1dtv)
    out = v - M.o1dtv
    sleep(1e-2) 
    return out
end

# overwrite `out` by `(I - P) * v`
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        Mt  :: Transpose{T, PageRankImPt{TA, IA, T}}, 
        v   :: Vector{T}) where {TA<:Number, IA<:Integer, T <: AbstractFloat}
    M = Mt.parent
    mul!(M.Av, M.A, v)
    onetv = dot(M.one, v)
    M.DAv .= M.Av .* M.d
    M.ztonetv .= M.z .* onetv
    axpby!(-1, M.DAv, -1, M.ztonetv)
    out = v - M.ztonetv
    return out
end

In [8]:
using DelimitedFiles

isfile("pgrksol.csv") || download("https://raw.githubusercontent.com/ucla-biostat-257-2020spring/ucla-biostat-257-2020spring.github.io/master/hw/hw3/pgrksol.csv", "./pgrksol.csv")
xsol = vec(readdlm("./pgrksol.csv"))

916428-element Array{Float64,1}:
 3.3783428216975054e-5
 2.0710155392568165e-6
 3.663065984832893e-6 
 7.527510785028837e-7 
 8.63328599674051e-7  
 1.769418252415541e-6 
 2.431230382883396e-7 
 6.368417180141445e-7 
 4.744973703681939e-7 
 2.6895486110647536e-7
 3.18574314847409e-6  
 7.375106374416742e-7 
 2.431230382883396e-7 
 ⋮                    
 1.1305006040148547e-6
 4.874825281822915e-6 
 3.167946973112519e-6 
 9.72688040308568e-7  
 6.588614479285245e-7 
 7.737011774300648e-7 
 2.431230382883396e-7 
 1.6219204214797293e-6
 3.912130060551738e-7 
 2.431230382883396e-7 
 7.296033831163157e-6 
 6.330939996912478e-7 

### Step 2
We want to benchmark the hot functions `mul!` to make sure they are efficient and allocate litte memory.

In [17]:
M = PageRankImPt(A, 0.85)
n = size(M, 1)
v, out = ones(n), zeros(n)

norm(transpose(M) * ones(n)) < 1e-12

false

In [ ]:
bm_mtv = @benchmark mul!($out, $(transpose(M)), $v) setup=(fill!(out, 0); fill!(v, 1))

In [ ]:
clamp(10 - median(bm_mv).memory / 100, 0, 10) + 
clamp(10 - median(bm_mtv).memory / 100, 0, 10)

In [14]:
M = PageRankImPt(A, 0.85)
n = size(M, 1)

@assert transpose(M) * ones(n) ≈ zeros(n)

AssertionError: AssertionError: transpose(M) * ones(n) ≈ zeros(n)

### Step 3
Let's first try to solve the PageRank problem by the GMRES method for solving linear equations.

In [ ]:
using KrylovKit

# normalize in-degrees to be the start point
x0   = vec(sum(A, dims = 1)) .+ 1.0
x0 ./= sum(x0)

# right hand side
b = zeros(n)

# warm up (compilation)
linsolve(M, b, x0, issymmetric = false, isposdef = false, maxiter = 1) 
# output is complex eigenvalue/eigenvector
(x_gmres, info), time_gmres, = @timed linsolve(M, b, x0, issymmetric = false, isposdef = false)

In [ ]:
@assert norm(x_gmres - xsol) < 1e-8

In [ ]:
clamp(20 / time_gmres * 20, 0, 20)

### Step 4

Let's first try to solve the PageRank problem by the Arnoldi method for solving eigen problems.

In [ ]:
# warm up (compilation)
eigsolve(M, x0, 1, :SR, issymmetric = false, maxiter = 1)
# output is complex eigenvalue/eigenvector
(vals, vecs, info), time_arnoldi, = @timed eigsolve(M, x0, 1, :SR, issymmetric = false)

In [ ]:
@assert abs(Real(vals[1])) < 1e-8

In [ ]:
x_arnoldi   = abs.(Real.(vecs[1]))
x_arnoldi ./= sum(x_arnoldi)
@assert norm(x_arnoldi - xsol) < 1e-8

In [ ]:
clamp(20 / time_arnoldi * 20, 0, 20)

## Question 6: Results

The top twenty pages based on this method is:

In [ ]:
sortperm(vec(x_arnoldi), rev = true)[1:20]

Versus

In [ ]:
sortperm(vec(x), rev = true)[1:20]